# 2-Step RAG 파이프라인 테스트

이 노트북은 다음과 같은 2단계 RAG(Retrieval-Augmented Generation) 프로세스를 테스트합니다.

1.  **1단계: 헤더 검색 (정책 식별)**
    *   사용자의 복합적인 질문을 `header_db` 벡터 저장소에 질의하여 가장 적합한 정책을 식별합니다.
    *   식별된 정책의 메타데이터에서 본문(body) 데이터가 저장된 컬렉션 이름(`original_collection_name`)을 추출합니다.

2.  **2단계: 본문 검색 (상세 정보 추출)**
    *   1단계에서 얻은 컬렉션 이름을 사용해 `body_db`에서 해당 정책의 상세 내용이 담긴 벡터 저장소를 로드합니다.
    *   '필수 조건', '혜택' 등 구체적인 키워드로 상세 정보를 검색합니다.

3.  **답변 생성**
    *   2단계에서 추출한 상세 정보들을 종합하여 LLM에게 전달하고, 사용자의 질문에 맞는 최종 답변을 생성합니다.

## 1. 환경 설정

In [1]:
import os
import json
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 기본 경로 설정
HEADER_DB_PATH = "../source/vectorstore/header_db"
BODY_DB_PATH = "../source/vectorstore/body_db"

# 모델 및 임베딩 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(
    model_name = "gpt-4.1-mini",
    temperature=0
)

## 2. 1단계: 헤더 검색 (정책 식별)

In [21]:
# 사용자 질문 정의
user_query = "나 지금 서울시에 거주중인데, 난 27세이고, 월세 보증금 지원을 받고 싶어, 이에 대한 정보를 알려줘"

# 헤더 벡터 저장소 로드
header_vectorstore = Chroma(
    persist_directory = HEADER_DB_PATH,
    embedding_function = embeddings,
    collection_name = "header_db"
)

# 헤더 검색
header_retriever = header_vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})
retrieved_headers = header_retriever.get_relevant_documents(user_query)

if not retrieved_headers:
    print("관련된 정책을 찾지 못했습니다.")
else:
    # 검색된 헤더 정보 파싱
    header_content = json.loads(retrieved_headers[0].page_content)
    
    policy_name = header_content.get("policy_name")
    original_collection_name = header_content.get("original_collection_name")

    print(f"[1단계 결과] 가장 관련성 높은 정책을 찾았습니다.")
    print(f"- 정책 이름: {policy_name}")
    print(f"- 상세 정보 컬렉션: {original_collection_name}")

[1단계 결과] 가장 관련성 높은 정책을 찾았습니다.
- 정책 이름: 서울시 청년월세지원정책
- 상세 정보 컬렉션: seoulsiceongnyeonweolsejiweonjeongcaeg_body


In [10]:
header_content

{'summary': {'policy_category': '주거',
  'policy_name': '2025년 서울시 청년월세지원 사업',
  'policy_target': '서울시 거주 청년 (19세 ~ 39세)',
  'policy_benefit': '월 최대 20만원, 최대 12개월간 지원 (총 240만원)'},
 'policy_name': '서울시 청년월세지원정책',
 'original_collection_name': 'seoulsiceongnyeonweolsejiweonjeongcaeg_body'}

## 3. 2단계: 본문 검색 (상세 정보 추출)

In [22]:
retrieved_body_chunks = []
if original_collection_name:
    # 본문 벡터 저장소 로드
    body_vectorstore = Chroma(
        persist_directory=BODY_DB_PATH,
        embedding_function=embeddings,
        collection_name=original_collection_name
    )
    
    # 상세 정보 키워드 리스트
    detail_queries = ["필수 조건", "우대 조건", "혜택", "필수 서류", "정책 지역", "제외 대상자"]
    
    # 검색된 문서의 ID를 저장하여 중복 방지
    retrieved_doc_ids = set()
    
    print(f"[2단계 시작] '{policy_name}' 정책의 상세 정보를 각 항목별로 검색합니다.")
    
    for query in detail_queries:
        # 유사도 점수 임계값을 사용한 리트리버 설정
        # score_threshold: 0.7 (0~1 사이, 1에 가까울수록 유사)
        # 여기서는 유사도 세팅값을 되게 낮춰야 좀 적당히 나오더라구요. 실 사용시에 여러번 낮은 값으로 테스트 해보시면 좋을 것 같습니다.
        # 어차피 정확한 하나의 문서에서 RAG 연산 할거라서 지금 보니까 임계값 없어도 될 것 같기도 하고..
        body_retriever = body_vectorstore.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={
                "k": 4,
                "score_threshold": 0.01
            }
        )
        
        # 각 키워드로 문서 검색
        retrieved_docs = body_retriever.get_relevant_documents(query)
        
        # 중복되지 않은 문서만 추가
        for doc in retrieved_docs:
            if doc.metadata.get('id', doc.page_content) not in retrieved_doc_ids:
                retrieved_body_chunks.append(doc)
                retrieved_doc_ids.add(doc.metadata.get('id', doc.page_content))
        
        print(f"- '{query}' 검색 완료 (유사도 0.7 이상, 최대 3개)")

    print(f"\\n[2단계 결과] 총 {len(retrieved_body_chunks)}개의 고유한 정보 조각을 찾았습니다.")

else:
    print("[2단계 결과] 상세 정보를 찾을 수 없습니다.")

[2단계 시작] '서울시 청년월세지원정책' 정책의 상세 정보를 각 항목별로 검색합니다.
- '필수 조건' 검색 완료 (유사도 0.7 이상, 최대 3개)


c:\POTENUP\10_LLM_TeamProject\.venv\Lib\site-packages\langchain_core\vectorstores\base.py:1070: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'creationdate': '2025-06-02T15:06:47+09:00', 'source': 'source\\original_docs\\서울시 청년월세지원정책.pdf', 'page': 6, 'producer': 'Microsoft: Print To PDF', 'title': '〲㔲⁄렜⃜䒭㣔타ꠠ⃑\ue0f5⸸睨硰', 'author': '', 'moddate': '2025-06-02T15:06:47+09:00', 'creator': 'PyPDF', 'total_pages': 10, 'page_label': '7'}, page_content='- 7 -\n     전대차 계약인 경우 아래 모두 제출3) , , ① ② ③ 전대차 계약서       ①       전대차 계약사항 전대인과 전차인의 성명 생년월일 도장 또는 서명 날인   ( ) , , ,※ 임차보증금과 월세금액 계약일자 소재지 등이 기재되어 있어야 함, , 전대인의 임대차 계약서       ② 건축물대장 건축물 현황도       , ③       건축물대장 등 해당 주택의 내부구조 파악이 가능한 자료   ※  ㅇ 이체확인증 : 신청일 기준 최근 개월3월(’25.3~5 )간 월세 이체 내역임대인 계좌 확인( )   - 이체 일자, 임대인 성명 또는 임대인 계좌번호, 월세 금액이 명시되어야 함   - 월세 받는 사람이 계약서상 임대인과 다를 경우 특약사항에 실제 월세를 지급받는 , 사람의 성함 계좌번호 명시되어야 함,    - 최근 개월 월세 이체 내역을 제출하기 어려운 경우 다음의 방법으로 제출3 , 임대차 계약 체결 후 차임 지급 기일이 도래하지 않아 월세 이체내역이 없는 경우       ,

- '우대 조건' 검색 완료 (유사도 0.7 이상, 최대 3개)


c:\POTENUP\10_LLM_TeamProject\.venv\Lib\site-packages\langchain_core\vectorstores\base.py:1070: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'creationdate': '2025-06-02T15:06:47+09:00', 'source': 'source\\original_docs\\서울시 청년월세지원정책.pdf', 'creator': 'PyPDF', 'producer': 'Microsoft: Print To PDF', 'page': 4, 'page_label': '5', 'author': '', 'total_pages': 10, 'title': '〲㔲⁄렜⃜䒭㣔타ꠠ⃑\ue0f5⸸睨硰', 'moddate': '2025-06-02T15:06:47+09:00'}, page_content='- 5 -\n사업 신청 제외 대상자 ◆  ㅇ 국토교통부 청년월세 한시 특별지원금을 수혜 중인 사람신청일 전 지원 종료 시 신청 가능   ※  ㅇ 서울시 청년월세지원금 기선정자로 수급 이력이 있는 사람  ㅇ 신청자 본인 주택 소유자( ) 주거용 오피스텔 분양권 입주권 공유지분 소유 등 포함( , , , )주택 청약 시 적용되는 주택소유여부 판단기준 과 관계없이 소유권 확인될 시 선정 불가   “ ”※  ㅇ 신청자 본인 일반재산 총액 억 천만원을 초과하는 사람( ) 1 3포함항목 토지과세표준액 건축물과세표준액 임차보증금 차량시가표준액    - ( ) , , ,  ㅇ 신청자 본인 차량시가표준액이 만원 이상의 자동차를 소유한 사람( ) 2,500 ㅇ 신청자 본인 국민기초생활수급을 받고 있는 사람( ) 생계 의료 주거급여 대상자( , , ) ㅇ 서울시 청년수당을 받고 있는 사람 년 자치구 자체 청년월세지원 사업, ’25은평 광진 등( , )지원자로 선정된 사람 자립준비청년 월세기숙사비 지원 등 , ․유사 사업 수혜를 받고

- '혜택' 검색 완료 (유사도 0.7 이상, 최대 3개)
- '필수 서류' 검색 완료 (유사도 0.7 이상, 최대 3개)
- '정책 지역' 검색 완료 (유사도 0.7 이상, 최대 3개)
- '제외 대상자' 검색 완료 (유사도 0.7 이상, 최대 3개)
\n[2단계 결과] 총 6개의 고유한 정보 조각을 찾았습니다.


## 4. 답변 생성

In [23]:
if retrieved_body_chunks:
    # 프롬프트 템플릿 정의
    template = """
    당신은 정책 전문가입니다. 다음은 '{policy_name}' 정책에 대한 정보입니다. 
    이 정보를 바탕으로 사용자의 질문에 맞춰 다음 6가지 항목에 대해 명확하고 간결하게 요약하여 답변해주세요.

    
    
    1.  **필수 조건**: 누가 신청할 수 있나요?
    2.  **우대 조건**: 어떤 경우에 더 유리한가요? (정보가 없다면 '해당 없음'으로 표시)
    3.  **혜택**: 무엇을 받을 수 있나요?
    4.  **필수 서류**: 무엇을 준비해야 하나요?
    5.  **정책 지역**: 어디에서 시행되는 정책인가요?
    6.  **제외 대상자**: 어떤 경우에 해당정책에 제외 되나요?
    --- 정보 ---
    {context}
    --- --- ---

    답변:
    """
    
    prompt = ChatPromptTemplate.from_template(template)
    
    # 검색된 정보 조각들을 하나의 문자열로 합침
    context_str = "\n\n".join([chunk.page_content for chunk in retrieved_body_chunks])
    
    # LangChain 체인 구성
    rag_chain = (
        {"context": lambda x: context_str, "policy_name": lambda x: policy_name} 
        | prompt 
        | llm 
        | StrOutputParser()
    )

    # 답변 생성 실행
    print("[3단계 결과] 최종 답변을 생성합니다.\n")
    final_answer = rag_chain.invoke({})
    print(final_answer)
else:
    print("[3단계 결과] 답변을 생성할 정보가 부족합니다.")

[3단계 결과] 최종 답변을 생성합니다.

1. **필수 조건**  
- 만 19세 이상 39세 이하 무주택자 청년  
- 서울시에 주민등록이 되어 있고 실제 거주하는 자  
- 신청인 가구의 건강보험료 부과액이 기준 중위소득 150% 이하  
- 임대차계약서상 임차인 본인이 신청해야 하며, 주민등록 전입신고가 되어 있어야 함  
- 외국인은 신청 불가, 재외국민은 건강보험 가입자 또는 피부양자일 경우 가능  

2. **우대 조건**  
- 해당 없음 (우대 조건에 관한 별도 정보 없음)  

3. **혜택**  
- 월세 지원금 지급 (임대차계약서에 명시된 월세 기준)  
- 선정인원 15,000명, 선정 후 격월로 계좌 입금 (예: 7~9월분 10월 지급, 10~11월분 12월 지급)  
- 임차보증금과 월세를 환산하여 지원금 산정  

4. **필수 서류**  
- 확정일자가 날인된 임대차계약서 전체 사본 (임대인, 임차인 성명, 생년월일, 임대차 기간, 보증금, 월세 명시)  
- 건강보험료 부과 내역 (자동 조회, 어려울 경우 별도 소득증빙 요청 가능)  
- 월세 이체 확인증 (최근 3개월간 월세 납부 내역)  
- 가족관계증명서 (본인 기준, 공고일 이후 발급분)  
- 전대차 계약 시 전대차 계약서, 전대인 임대차 계약서, 건축물대장 등 추가 서류  
- 고시원, 게스트하우스 등 특수 주거형태는 입실확인서, 임대사업자등록증 등 별도 제출  

5. **정책 지역**  
- 서울특별시 내에서 시행  

6. **제외 대상자**  
- 국토교통부 청년월세 한시 특별지원금 수혜 중인 자 (지원 종료 후 신청 가능)  
- 서울시 청년월세지원금 기선정자  
- 신청인 본인 주택 소유자(분양권, 입주권, 공유지분 포함)  
- 일반재산 총액 1억 3천만원 초과자  
- 차량 시가표준액 2,500만원 초과 자동차 소유자  
- 국민기초생활수급자(생계, 의료, 주거급여 대상자)  
- 서울시 청년수당 수혜자 및 유사 자치구 청년월세지원 사업 수혜자  
- 